In [ ]:
import os
import numpy as np
from context import src, utils
from utils.rf_plot import show_fields
from src.analyzer import DataAnalyzer
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
output_dir = '../output/motion_benefit4'

plt.rcParams['font.size'] = 7

from matplotlib.ticker import MaxNLocator

In [ ]:
# output_dir = 'output/sparsity'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.h5')]
pkl_fns.sort()
len(pkl_fns)

In [ ]:
# import pickle as pkl
# from utils.h5py_utils import save_dict

# pkl_fns = [os.path.join(output_dir, fn) 
#            for fn in os.listdir(output_dir) 
#            if fn.endswith('.pkl')]
# pkl_fns.sort()
# len(pkl_fns)

# for pkl_fn in pkl_fns:
#     print pkl_fn
#     with open(pkl_fn, 'rb') as f:
#         data = pkl.load(f)
#     save_fn = pkl_fn.replace('pkl', 'h5')
#     save_dict(save_fn, data)

In [ ]:
da_ = [DataAnalyzer.fromfilename(pkl_fn) for pkl_fn in pkl_fns[21:25]]

In [ ]:
da = da_[1]

In [ ]:
# fp = da.data['FP']

# plt.hist(fp.ravel(), bins=50)
# plt.yscale('log')

In [ ]:
# ax = plt.axes()
# ips = da.data['Ips'][0, :, 50]
# xe = da.data['XE']
# ye = da.data['YE']
# ie = da.data['IE']
# de = da.data['de']

# m = abs(ips).max()

# for x, y, i, val in zip(xe, ye, ie, ips):
#     ax.add_patch(plt.Circle((x, -y), de * 0.203, alpha=val/m))

# rng = max(max(xe), max(ye))
# ax.set_xlim([-rng, rng])
# ax.set_ylim([-rng, rng])
# ax.set_aspect('equal')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=False,
                         figsize=(3, 1.2))

for i, q in enumerate([9, 69, 139]):
    ax = axes[i]
    colorbar = i == 2
    colorbar = False
    da.plot_image_estimate(fig, ax, q=q,
                           colorbar=colorbar)
    t = da.data['EM_data/{}/time_steps'.format(q)]
    ax.set_title('T = {} ms'.format(t), fontsize=7)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)


plt.tight_layout()

plt.savefig('../output/improvement.pdf')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharey=True,
                         figsize=(2.1, 1))

for i in range(2):
    ax = axes[i]
    da.plot_path_estimate(ax, q, i)
    ax.set_title('')
    ax.set_ylabel('')
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks([0, .7])
    ax.set_xlabel('')
axes[0].yaxis.set_ticks([-6, 6])
plt.tight_layout()
plt.savefig('../output/path.pdf')

In [ ]:
fig, axes = da.plot_em_estimate(139, figsize=(3.5, 5))

axes[2][1].set_ylabel('')


axes[0][0].set_ylabel('Position (arcmin)')
axes[1][0].set_ylabel('Position (arcmin)')

start, stop = axes[0][1].get_xlim()
ticks = np.arange(start, stop + 0.5, 2)
axes[0][1].set_xticks(ticks)
axes[0][1].set_yticks(ticks)

plt.tight_layout()
plt.savefig('../output/example.pdf')
# for _ in axes.flat:
#     pass

In [ ]:
fig, axes = da_[0].plot_em_estimate(139, figsize=(3.5, 5.5))

plt.suptitle('asdfadfasdfhi')

axes[2][1].set_ylabel('')


axes[0][0].set_ylabel('Position (arcmin)')
axes[1][0].set_ylabel('Position (arcmin)')

start, stop = axes[0][1].get_xlim()
ticks = np.arange(start, stop + 0.5, 2)
axes[0][1].set_xticks(ticks)
axes[0][1].set_yticks(ticks)


plt.tight_layout()
# plt.savefig('output/example.pdf')
# for _ in axes.flat:
#     pass

In [ ]:
fig, ax = plt.subplots(figsize=(0.75, 0.75))
ax.set_xlabel(None, fontsize=7)
ax.set_ylabel(None, fontsize=7)

ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')



# ax.get_xaxis().set_ticks([])
start, stop = ax.get_xlim()
xres = 0.5
ticks = np.arange(start, stop + xres, xres)
ax.set_xticks(ticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(7)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(7)

    
da.plot_tuning_curves()

start, stop = ax.get_ylim()
xres = 40
ticks = [10, 50, 100]
ax.set_yticks(ticks)

ax.set_ylabel('')
ax.set_xlabel('')

ax.legend().set_visible(False)
ax.title.set_fontsize(7)
plt.tight_layout(pad=0.3)
plt.savefig('../output/firing_rates.pdf')

In [ ]:
save_dir = '../output/spike_video/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
%%time
da.plot_image_and_rfs(alpha_rf=0.25)

In [ ]:
for i, q in enumerate(range(0, da.N_T, 1)):
# for i, q in enumerate(range(0, 1, 1)):
    print 'Creating figure {:04d}'.format(q)
    da.plot_moving_image_and_spikes(q)
    plt.suptitle('Image Projected on the Retina and Generated Spikes at t = {:03d} ms'.format(q))
    plt.savefig(os.path.join(save_dir, 'img_{:04d}'.format(i)))
    plt.close()

To synthesize the different png files run:

`
avconv -framerate 20 -i img_%04d.png -c:v libx264 -r 30 video/out.mp4
`

In [ ]:
def plot_em_estimate(self, q, figsize=None):
    """Visualize the results after iteration q."""
    if q == -1:
        q = self.N_itr - 1

    n_time_steps = self.data['EM_data/{}/time_steps'.format(q)]
    t_ms = self.DT * n_time_steps * 1000.

    if figsize is None:
        figsize=(3.5, 5)
    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=figsize)
    #  fig.suptitle('EM Reconstruction after t = {} ms'.format(t_ms))

    self.plot_spikes(ax[0][1], n_time_steps - 1, mode='ON')

    self.plot_spikes(ax[1][1], n_time_steps - 1, mode='OFF')

    self.plot_image_estimate(fig, ax[1][0], q, cmap=plt.cm.gray_r)

    self.plot_image_and_rfs(fig, ax=ax[0][0], legend=False, q=n_time_steps - 1, alpha_rf=0.25)
    ax[0][0].set_title('Pattern with Cone RFs')

    self.plot_path_estimate(ax[0][2], q, 0)
    self.plot_path_estimate(ax[0][2], q, 1)

    for u in [0, 1]:
        ax_ = ax[u][2]
        ax_.xaxis.set_major_locator(MaxNLocator(2))
        
    u0, v0 = ax[0][2].get_ylim()
    u1, v1 = ax[1][2].get_ylim()

    for u in [0, 1]:
        ax[u][2].set_ylim(min(u0, u1), max(v0, v1))



#     axes[2][1].set_ylabel('')


    ax[0][0].set_ylabel('Position (arcmin)')
    ax[0][1].set_ylabel('Position (arcmin)')
    ax[1][1].set_ylabel('Position (arcmin)')

#     start, stop = axes[0][1].get_xlim()
#     ticks = np.arange(start, stop + 0.5, 2)
#     axes[0][1].set_xticks(ticks)
#     axes[0][1].set_yticks(ticks)


    plt.tight_layout()
    plt.subplots_adjust(top=0.85)

    return fig, ax

In [ ]:
for i, q in enumerate(range(0, da.N_T, 1)):
# for i, q in enumerate(range(2)):
    print 'Creating figure {:04d}'.format(q)    
    fig, axes = plot_em_estimate(da, q, figsize=(6, 4.5))
    plt.suptitle('Image Projected on the Retina and Generated Spikes at t = {:03d} ms'.format((q+1) * 5), fontsize=10)
    plt.savefig(os.path.join(save_dir, 'img_rec_{:04d}.png'.format(i)), dpi=200)
    plt.close()

r"""
avconv -framerate 20 -i img_rec_%04d.png -c:v libx264 -r 30 video/out_rec.mp4
"""


# Motion Prior

In [ ]:
output_dir = '../output/natural_sparsity_van_hateren3'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.pkl')]
pkl_fns.sort()
len(pkl_fns)

In [ ]:
da_ = [DataAnalyzer.fromfilename(pkl_fn) for pkl_fn in pkl_fns]

In [ ]:
da = da_[7]

In [ ]:
save_dir = '../output/natural_spike_video/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
A = da.data['EM_data/{}/coeff_est'.format(i)]
D = da.data['D']

In [ ]:
_ = plt.hist(A, bins=50)

In [ ]:
def equalize_ys(ax0, ax1):
    u0, v0 = ax0.get_ylim()
    u1, v1 = ax1.get_ylim()
    for ax in [ax0, ax1]:
        ax.set_ylim(min(u0, u1), max(v0, v1))

In [ ]:
def plot_em_estimate_dict(self, q, figsize=None):
    """Visualize the results after iteration q."""
    if q == -1:
        q = self.N_itr - 1

    n_time_steps = self.data['EM_data/{}/time_steps'.format(q)]
    t_ms = self.DT * n_time_steps * 1000.
#     fig, axes = plt.subplots(nrows=2, ncols=3, figsize=figsize)

    fig = plt.figure(figsize=(8, 4))
    shape = (4, 8)
    ax1 = plt.subplot2grid(shape, (0, 0), rowspan=2, colspan=2)
    ax2 = plt.subplot2grid(shape, (2, 0), rowspan=2, colspan=2)
    ax3 = plt.subplot2grid(shape, (2, 2), rowspan=2, colspan=2)
#     ax4 = plt.subplot2grid(shape, (3, 2), rowspan=1, colspan=2)

    ax5 = plt.subplot2grid(shape, (0, 2), rowspan=2, colspan=2)
    ax6 = plt.subplot2grid(shape, (0, 4), rowspan=4, colspan=4)

#     axes = [ax1, ax2, ax3, ax4, ax5]



    #  fig.suptitle('EM Reconstruction after t = {} ms'.format(t_ms))

    ax = ax5
    self.plot_spikes(ax, n_time_steps - 1, mode='ON')

#     self.plot_spikes(ax, n_time_steps - 1, mode='OFF')

    ax = ax2
    self.plot_image_estimate(fig, ax, q, cmap=plt.cm.gray_r, colorbar=False)
    
    ax = ax1
    self.plot_image_and_rfs(fig, ax=ax, legend=False, q=n_time_steps - 1, alpha_rf=0.25)
    ax.set_title('Pattern with Cone RFs')

    ax = ax3
    self.plot_path_estimate(ax, q, 0)
#     ax.set_xlabel('')
    ax.set_ylabel('')
#     ax.xaxis.set_visible(False)
    ax = ax3
    self.plot_path_estimate(ax, q, 1)
    ax.set_ylabel('')
    ax.set_title('X and Y Positions')
    
#     for ax in [ax3, ax4]:
    for ax in [ax3]:
        ax.xaxis.set_major_locator(MaxNLocator(2))
        
#     equalize_ys(ax3, ax4)


    ax = ax6

    A = self.data['EM_data/{}/coeff_est'.format(q)]
    D = self.data['D']
    sort_idx = np.argsort(np.abs(A))[::-1]
    n_active = np.sum(np.abs(A) > 0.5)

    active_idx = sort_idx[0:n_active]

    n_sp = D.shape[0]
#     ax.set_title('Histogram of sparse Coefficients: \n Percentage of active coefficients {:.0f}%'.format(
#             100. * n_active / n_sp))
#     ax.set_xlabel('Coefficient Activity')

    show_fields(D[active_idx] *
                A[active_idx][:, np.newaxis], fig=fig, ax=ax,
                cmap = plt.cm.gray, pos_only = False, colorbar=False)
    ax.set_title('Receptive Fields of Active Neurons \n Scaled by their Activations')
    

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)

    return fig, ax

In [ ]:
from time import time

In [ ]:
t0 = time()
for i in range(da.N_itr):
    t1, t0 = (time(), t1)
    print i, t1 - t0
    fig, axes = plot_em_estimate_dict(da, i, figsize=(6, 4.5))
    plt.suptitle('Pattern and Trajectory Reconstruction using Sparsely Activated Neurons at t = {:03d} ms'.format((i+1) * 5), fontsize=10)
    plt.savefig(os.path.join(save_dir, 'img_{:04d}'.format(i)), dpi=200)
    plt.close()

In [ ]:
r"""
mkdir video
avconv -framerate 10 -i img_%04d.png -c:v libx264 -r 30 video/sparse_rec.mp4
"""

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

i = 102

A = da.data['EM_data/{}/coeff_est'.format(i)]
D = da.data['D']

ax = axes[0]
ax.hist(A, bins=100)

sort_idx = np.argsort(np.abs(A))[::-1]
n_active = np.sum(np.abs(A) > 0.5)

active_idx = sort_idx[0:n_active]

n_sp = D.shape[0]
ax.set_title('Histogram of sparse Coefficients: \n Percentage of active coefficients {:.0f}%'.format(
        100. * n_active / n_sp))
ax.set_xlabel('Coefficient Activity')

ax = axes[1]
show_fields(D[active_idx] *
            A[active_idx][:, np.newaxis], fig=fig, ax=ax,
            cmap = plt.cm.gray, pos_only = False, colorbar=False)
ax.set_title('Active Dictionary Elements \n Scaled by their activations')